# Distilbert experimenty

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tensorflow as tf
from torch.utils.data import Dataset
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
from typing import Dict, List
import transformers
from transformers import AutoModel, BertTokenizerFast,BertTokenizer, DistilBertTokenizerFast
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM

!pip install transformers[torch]
!pip install accelerate -U
!pip install transformers==4.30

## Načítanie a príprava dát

In [ ]:
dataset = pd.read_csv('../Data/final_dataset_4.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['processed_text'], dataset['author_id'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_valid = label_encoder.transform(y_valid)
y_test = label_encoder.transform(y_test)

## Distilbert

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
x_train = list(X_train)
x_val = list(X_valid)
x_test = list(X_test)

y_train = list(y_train)
y_val = list(y_valid)
y_test = list(y_test)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val,truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
train_dataset = CustomDataset(train_encodings, y_train)
val_dataset = CustomDataset(val_encodings, y_valid)
test_dataset = CustomDataset(test_encodings, y_test)

In [ ]:
labels_count = len(dataset['author_id'].unique())
print(labels_count)

20


## Experiment 1 - learning_rate = 5e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=5e-5,             #default
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.we

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.989167,0.103093,0.149504,0.134167,0.100025
2,No log,2.831125,0.278351,0.227736,0.313333,0.242007
3,No log,2.303661,0.432990,0.365361,0.465476,0.386936
4,No log,1.842028,0.587629,0.617857,0.630952,0.575522
5,No log,1.473127,0.618557,0.631667,0.658333,0.608406
6,No log,1.186197,0.680412,0.707619,0.691786,0.659419
7,No log,0.956912,0.670103,0.690595,0.707619,0.670314
8,No log,1.075252,0.711340,0.737083,0.749762,0.702022
9,No log,0.977129,0.731959,0.742179,0.750595,0.714129
10,1.455400,0.936489,0.762887,0.724583,0.775238,0.727824


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=660, training_loss=1.1075369603706129, metrics={'train_runtime': 242.7179, 'train_samples_per_second': 43.062, 'train_steps_per_second': 2.719, 'total_flos': 1384993697218560.0, 'train_loss': 1.1075369603706129, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-2.3886719 , -1.6083984 , -0.40478516, ..., -1.7666016 ,
         4.9648438 , -1.6933594 ],
       [-1.2041016 , -1.0234375 , -0.86621094, ..., -1.4873047 ,
         2.1328125 ,  2.5429688 ],
       [-1.1875    , -1.1875    , -0.7265625 , ..., -0.78271484,
         0.75146484,  6.8515625 ],
       ...,
       [-1.5732422 , -1.8066406 , -0.4621582 , ..., -1.2050781 ,
         4.9648438 ,  3.4140625 ],
       [-1.4697266 , -0.64453125, -1.2773438 , ..., -0.6748047 ,
        -1.0058594 ,  1.1708984 ],
       [-1.9599609 , -1.6298828 , -0.7211914 , ..., -1.3134766 ,
         6.6835938 ,  0.3630371 ]], dtype=float32), label_ids=array([16,  3, 19, 16,  7, 18, 15,  4, 17,  2, 11,  5, 10,  0, 11, 16,  9,
       15,  9,  0, 10, 11,  5, 13,  8,  1, 10, 13,  0,  0,  8, 13,  8, 19,
       13,  2, 14,  4,  5,  0,  5,  7, 12, 10,  5, 16,  6, 16,  2,  1, 10,
        9, 12, 11,  6,  6,  4,  0,  3, 10,  4,  4, 19,  0, 13, 17, 13,  5,
        3,  7, 15, 19, 16,  2,  

## Experiment 4 - learning_rate = 8e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=8e-5,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.we

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.958723,0.134021,0.047156,0.118333,0.056780
2,No log,2.557084,0.278351,0.250490,0.326310,0.213109
3,No log,2.000649,0.525773,0.470671,0.549405,0.459083
4,No log,1.622498,0.463918,0.507837,0.517857,0.438885
5,No log,1.240805,0.639175,0.628869,0.652619,0.613657
6,No log,1.192315,0.649485,0.649444,0.632381,0.608211
7,No log,1.166811,0.628866,0.579091,0.619286,0.571717
8,No log,1.238935,0.690722,0.708175,0.724405,0.669026
9,No log,1.322750,0.659794,0.660833,0.683452,0.648125
10,1.218400,1.328662,0.711340,0.709583,0.729762,0.702525


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=660, training_loss=0.9247653184515057, metrics={'train_runtime': 234.3126, 'train_samples_per_second': 44.607, 'train_steps_per_second': 2.817, 'total_flos': 1384993697218560.0, 'train_loss': 0.9247653184515057, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-2.8144531 , -1.3662109 , -2.4082031 , ..., -2.7519531 ,
         6.6445312 , -0.9707031 ],
       [-0.11120605, -0.9448242 , -2.203125  , ..., -3.1230469 ,
         0.7055664 , -0.16992188],
       [-2.5039062 , -2.4492188 , -3.0722656 , ..., -1.4443359 ,
        -0.30200195,  7.0429688 ],
       ...,
       [-2.8808594 , -2.0800781 , -3.2675781 , ..., -2.5117188 ,
         3.5410156 ,  5.1601562 ],
       [-1.8984375 , -2.2246094 , -1.9824219 , ..., -0.3527832 ,
        -0.8364258 ,  0.7006836 ],
       [-2.7558594 , -1.5166016 , -3.1210938 , ..., -2.8085938 ,
         5.453125  , -0.7133789 ]], dtype=float32), label_ids=array([16,  3, 19, 16,  7, 18, 15,  4, 17,  2, 11,  5, 10,  0, 11, 16,  9,
       15,  9,  0, 10, 11,  5, 13,  8,  1, 10, 13,  0,  0,  8, 13,  8, 19,
       13,  2, 14,  4,  5,  0,  5,  7, 12, 10,  5, 16,  6, 16,  2,  1, 10,
        9, 12, 11,  6,  6,  4,  0,  3, 10,  4,  4, 19,  0, 13, 17, 13,  5,
        3,  7, 15, 19, 16,  2,  